In [1]:
HYDRA_FULL_ERROR=1
import hydra
from typing import Optional
from collections.abc import Callable
from omegaconf import OmegaConf, DictConfig

from hydra import initialize, initialize_config_module, initialize_config_dir, compose

from hydra.utils import instantiate
from torchmetrics import MetricCollection

        
def setup_config(cfg: DictConfig):
    '''
    # TODO default_config.yaml에서 save_dir: ${hydra:runtime.cwd}/logs/ resolve 할 때 error 생김
    # TODO resolve 하기 전에 save_dir key remove 해주는 코드 추가로 필요함
    '''
    OmegaConf.set_struct(cfg, False)
    OmegaConf.resolve(cfg)
    OmegaConf.set_struct(cfg, True)

In [2]:
# *  config setup  * #
CONFIG_PATH = '/usr/src/CV_For_Autonomous_Driving/config'

with initialize_config_dir(config_dir=CONFIG_PATH):
    cfg = compose(config_name='default_config_debug_sameD.yaml')

setup_config(cfg)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  after removing the cwd from sys.path.


## Load base_1e-3 experiment
### for debug metrics in 3 classes

In [3]:
import logging
from pathlib import Path

import torch
import hydra
import pytorch_lightning as pl

from scripts.utils import setup_config, setup_experiment, remove_prefix

log = logging.getLogger(__name__)

def resume_training(experiment):
    save_dir = Path(experiment.save_dir).resolve()
    checkpoints = list(save_dir.glob(f'**/{experiment.uuid}/checkpoints/*.ckpt'))

    print(f'**/{experiment.uuid}/checkpoints/*.ckpt')
    log.info(f'Searching {save_dir}.')

    if not checkpoints:
        return None

    log.info(f'Found {checkpoints[-1]}.')

    return checkpoints[-1]


pl.seed_everything(cfg.experiment.seed, workers=True)
Path(cfg.experiment.save_dir).mkdir(exist_ok=True, parents=False)

# Logger
logger = pl.loggers.WandbLogger(project=cfg.experiment.project,
                                save_dir=cfg.experiment.save_dir,
                                id=cfg.experiment.uuid)

# Create and load model/data
model_module, data_module = setup_experiment(cfg)
eval_loader = data_module.eval_dataloader()

# load model
ckpt_path = resume_training(cfg.experiment)
ckpt = torch.load(ckpt_path)
state_dict = remove_prefix(ckpt['state_dict'], 'fullmodel')

model_module.fullmodel.load_state_dict(state_dict)
model_module.to('cuda')

print('loaded pretrained network! Start Evaluation!')
print(len(eval_loader))

# evaluation (pl version)
#! only 5 Batch 
trainer = pl.Trainer(logger=logger, accelerator='gpu', gpus=[0],fast_dev_run=True)
trainer.test(model=model_module,
            dataloaders=eval_loader)


Global seed set to 2022


Loaded pretrained weights for efficientnet-b4
Loading NuScenes tables for version v1.0-trainval...


/opt/conda/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (IoUMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 35.484 seconds.
Reverse indexing ...
Done reverse indexing in 11.3 seconds.
**/0103_170521/checkpoints/*.ckpt


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


loaded pretrained network! Start Evaluation!
6019


test/metrics/lane_iou ('lane_iou', 0.016313213855028152)
test/metrics/road_iou ('road_iou', 0.6495959162712097)
test/metrics/vehicle_iou ('vehicle_iou', 0.0)
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/loss/center': 0.0005716345040127635,
 'test/loss/lane': 0.010303634218871593,
 'test/loss/road': 0.03855156898498535,
 'test/loss/vehicle': 0.011755327694118023,
 'test/total_loss': 0.021333962678909302}
--------------------------------------------------------------------------------



[{'test/total_loss': 0.021333962678909302,
  'test/loss/road': 0.03855156898498535,
  'test/loss/lane': 0.010303634218871593,
  'test/loss/vehicle': 0.011755327694118023,
  'test/loss/center': 0.0005716345040127635}]

In [4]:
metrics = {k: v for k, v in instantiate(cfg.metrics).items()}
metrics_collections = MetricCollection(metrics)

/opt/conda/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (IoUMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


In [5]:
for k, v in metrics_collections.items():
    print(k, v.label_indices)

lane_iou [[2, 3]]
road_iou [[0, 1]]
vehicle_iou [[4, 5, 6, 7, 8, 10, 11]]


In [6]:

prefix = 'train'

# for _, metrics_collection in metrics_collections.items():
#     metrics = metrics_collection.compute()
#     for k, v in metrics.items():
#         print(f'{prefix}/metrics/{k}', v)

metrics = metrics_collections.compute()
print(metrics)
for key, value in metrics.items():
    if isinstance(value, dict):
        for subkey, val in value.items():
            print(f'{prefix}/metrics/{key}{subkey}', val)
    else:
        print(f'{prefix}/metrics/{key}', value)

{'lane_iou': ('lane_iou', 0.0), 'road_iou': ('road_iou', 0.0), 'vehicle_iou': ('vehicle_iou', 0.0)}
train/metrics/lane_iou ('lane_iou', 0.0)
train/metrics/road_iou ('road_iou', 0.0)
train/metrics/vehicle_iou ('vehicle_iou', 0.0)


/opt/conda/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric IoUMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


In [ ]:
train/metrics/@0.40 0.0
train/metrics/@0.50 0.0

train/metrics/road_iou 0.0

train/metrics/road_iou IoUMetric()
train/metrics/lane_iou IoUMetric()
train/metrics/vehicle_iou IoUMetric()

In [6]:
from model_module.losses import MultipleLoss
loss_func = MultipleLoss(instantiate(cfg.loss))

In [7]:
for k, v in loss_func.items():
    print(k, v)

road BinarySegmentationLoss()
lane BinarySegmentationLoss()
vehicle BinarySegmentationLoss()
center CenterLoss()


In [8]:
import torch
metrics_no_g = MetricCollection({k: v for k, v in instantiate(cfg.metrics).items()})
metrics = MetricCollection({k: v for k, v in instantiate(cfg.metrics).items()},compute_groups=[['road_iou'], ['lane_iou'], ['vehicle_iou']])

print('metrics',metrics.compute())
print('metrics group:',metrics.compute_groups)
print('metrics_no_g',metrics_no_g.compute())
print('metrics_no_g group:',metrics_no_g.compute_groups)
print()
print('update')
print()
metrics.update()
metrics_no_g.update()
print('metrics',metrics.compute())
print('metrics group:',metrics.compute_groups)
print('metrics_no_g',metrics_no_g.compute())
print('metrics_no_g group:',metrics_no_g.compute_groups)

metrics {'lane_iou': ('lane_iou', 1.0), 'road_iou': ('road_iou', 0.0), 'vehicle_iou': ('vehicle_iou', 2.0)}
metrics group: {0: ['road_iou'], 1: ['lane_iou'], 2: ['vehicle_iou']}
metrics_no_g {'lane_iou': ('lane_iou', 1.0), 'road_iou': ('road_iou', 0.0), 'vehicle_iou': ('vehicle_iou', 2.0)}
metrics_no_g group: {0: ['lane_iou'], 1: ['road_iou'], 2: ['vehicle_iou']}

update

metrics {'lane_iou': ('lane_iou', 1.0), 'road_iou': ('road_iou', 0.0), 'vehicle_iou': ('vehicle_iou', 2.0)}
metrics group: {0: ['road_iou'], 1: ['lane_iou'], 2: ['vehicle_iou']}
metrics_no_g {'lane_iou': ('lane_iou', 1.0), 'road_iou': ('road_iou', 0.0), 'vehicle_iou': ('vehicle_iou', 2.0)}
metrics_no_g group: {0: ['lane_iou', 'road_iou', 'vehicle_iou']}
